In [42]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

In [43]:
def oracle(answer: str, answer_start: int, document: str) -> bool:
    """Returns True if the answer is in the document at the given index."""
    return answer_start != -1 and document[answer_start:answer_start + len(answer)] == answer

In [44]:
arab_train = train_set.filter(lambda example: example['language'] == 'arabic')
arab_validation = validation_set.filter(lambda example: example['language'] == 'arabic')

bengali_train = train_set.filter(lambda example: example['language'] == 'bengali')
bengali_validation = validation_set.filter(lambda example: example['language'] == 'bengali')

indonesian_train = train_set.filter(lambda example: example['language'] == 'indonesian')
indonesian_validation = validation_set.filter(lambda example: example['language'] == 'indonesian')

In [45]:
import pandas as pd
import numpy as np
arab_train_df = pd.DataFrame(arab_train)
arab_validation_df = pd.DataFrame(arab_validation)

bengali_train_df = pd.DataFrame(bengali_train)
bengali_validation_df = pd.DataFrame(bengali_validation)

indonesian_train_df = pd.DataFrame(indonesian_train)
indonesian_validation_df = pd.DataFrame(indonesian_validation)


In [46]:
arab_trainDf_final = arab_train_df[['question_text', 'document_plaintext']]
arab_validationDf_final = arab_validation_df[['question_text', 'document_plaintext']]

bengali_trainDf_final = bengali_train_df[['question_text', 'document_plaintext']]
bengali_validationDf_final = bengali_validation_df[['question_text', 'document_plaintext']]

indonesian_trainDf_final = indonesian_train_df[['question_text', 'document_plaintext']]
indonesian_validationDf_final = indonesian_validation_df[['question_text', 'document_plaintext']]



In [47]:
arab_trainDf_final['oracle_result']  = arab_train_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answer_start'][0]), row['document_plaintext']), axis=1)
arab_validationDf_final['oracle_result']  = arab_validation_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answer_start'][0]), row['document_plaintext']), axis=1)

bengali_trainDf_final['oracle_result']  = bengali_train_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answer_start'][0]), row['document_plaintext']), axis=1)
bengali_validationDf_final['oracle_result']  = bengali_validation_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answer_start'][0]), row['document_plaintext']), axis=1)

indonesian_trainDf_final['oracle_result']  = indonesian_train_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answer_start'][0]), row['document_plaintext']), axis=1)
indonesian_validationDf_final['oracle_result']  = indonesian_validation_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answer_start'][0]), row['document_plaintext']), axis=1)

<ipython-input-47-1b16c5326dc2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arab_trainDf_final['oracle_result']  = arab_train_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answer_start'][0]), row['document_plaintext']), axis=1)
<ipython-input-47-1b16c5326dc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arab_validationDf_final['oracle_result']  = arab_validation_df.apply(lambda row: oracle(row['annotations']['answer_text'][0], int(row['annotations']['answ

In [48]:
arab_trainDf_final.head()

,question_text,document_plaintext,oracle_result
0,ما هي المسألة الشرقية ؟,\n\nالمسألة الشرقية (بالإنجليزية: Eastern Ques...,True
1,كم تبعد بيت لحم على القدس؟,بيت لحم (بالسريانية: ܒܝܬ ܠܚܡ ؛ باليونانية: Βηθ...,True
2,متى ظهرت جماعة الاخوان المسلمين في سوريا؟,\nالإخوان المسلمون في سوريا: أسسها طلاب في ثلا...,True
3,متى تاسست السلفية الجهادبة في مصر ؟,نشأت الدعوة السلفية بالإسكندرية في سبعينيات ال...,True
4,من هو ريتشارد دوكينز؟,\nكلينتون ريتشارد دوكينز عالمُ سلوك حيوان ، و ...,True


# Data Preprocessing


In [49]:
import re
import string

def custom_standardization(input_data):
    lowercase = input_data.lower()
    lowercase = re.sub('\[\d+\]', '', lowercase)
    lowercase = re.sub('\n', ' ', lowercase)
    lowercase = re.sub('[%s]' % re.escape(string.punctuation), '', lowercase)
    return lowercase

In [50]:
arab_validationDf_final['document_plaintext'] [18]

'\n\nكاثرين صوفي فاينر (من مواليد يناير 1971[1][2]) هي صحفية وكاتبة مسرحية بريطانية. في 1 يونيو 2015 أصبحت أول امرأة تتقلد منصب رئيسة تحرير صحيفة الغارديان خلفًا لآلان روسبيرجير[3][4] بعد أن ترأست سابقا عمليات الويب الخاصة بالغارديان في أستراليا والولايات المتحدة، قبل أن يتم اختيارها لمنصب رئيس تحرير الجريدة العريقة[5].'

In [51]:
arab_trainDf_final['question_text']  = arab_trainDf_final['question_text'].apply(custom_standardization)
arab_trainDf_final['document_plaintext']  = arab_trainDf_final['document_plaintext'].apply(custom_standardization)

arab_validationDf_final['question_text']  = arab_validationDf_final['question_text'].apply(custom_standardization)
arab_validationDf_final['document_plaintext']  = arab_validationDf_final['document_plaintext'].apply(custom_standardization)

<ipython-input-51-2dda9d169521>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arab_trainDf_final['question_text']  = arab_trainDf_final['question_text'].apply(custom_standardization)
<ipython-input-51-2dda9d169521>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arab_trainDf_final['document_plaintext']  = arab_trainDf_final['document_plaintext'].apply(custom_standardization)
<ipython-input-51-2dda9d169521>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [52]:
bengali_trainDf_final['question_text']  = bengali_trainDf_final['question_text'].apply(custom_standardization)
bengali_trainDf_final['document_plaintext']  = bengali_trainDf_final['document_plaintext'].apply(custom_standardization)

bengali_validationDf_final['question_text']  = bengali_validationDf_final['question_text'].apply(custom_standardization)
bengali_validationDf_final['document_plaintext']  = bengali_validationDf_final['document_plaintext'].apply(custom_standardization)

<ipython-input-52-6ec9ede86076>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bengali_trainDf_final['question_text']  = bengali_trainDf_final['question_text'].apply(custom_standardization)
<ipython-input-52-6ec9ede86076>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bengali_trainDf_final['document_plaintext']  = bengali_trainDf_final['document_plaintext'].apply(custom_standardization)
<ipython-input-52-6ec9ede86076>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [53]:
indonesian_trainDf_final['question_text']  = indonesian_trainDf_final['question_text'].apply(custom_standardization)
indonesian_trainDf_final['document_plaintext']  = indonesian_trainDf_final['document_plaintext'].apply(custom_standardization)

indonesian_validationDf_final['question_text']  = indonesian_validationDf_final['question_text'].apply(custom_standardization)
indonesian_validationDf_final['document_plaintext']  = indonesian_validationDf_final['document_plaintext'].apply(custom_standardization)

<ipython-input-53-c3d8215d4682>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indonesian_trainDf_final['question_text']  = indonesian_trainDf_final['question_text'].apply(custom_standardization)
<ipython-input-53-c3d8215d4682>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indonesian_trainDf_final['document_plaintext']  = indonesian_trainDf_final['document_plaintext'].apply(custom_standardization)
<ipython-input-53-c3d8215d4682>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

# Stop words removal

In [54]:
from collections import Counter
import nltk
nltk.download('punkt')


arabic_word_counts = arab_trainDf_final['document_plaintext'].str.split(expand=True).stack().value_counts()
bengali_word_counts = bengali_trainDf_final['document_plaintext'].str.split(expand=True).stack().value_counts()
indonesian_word_counts = indonesian_trainDf_final['document_plaintext'].str.split(expand=True).stack().value_counts()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
stopwords_N = 20
arabic_stopwords = arabic_word_counts.head(stopwords_N).index.to_list()
bengali_stopwords = bengali_word_counts.head(stopwords_N).index.to_list()
indonesian_stopwords = indonesian_word_counts.head(stopwords_N).index.to_list()

# tf-idf vectorizer

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
arab_vectorizer = TfidfVectorizer(stop_words = arabic_stopwords)
bengali_vectorizer = TfidfVectorizer(stop_words = bengali_stopwords)
indonesian_vectorizer = TfidfVectorizer(stop_words = indonesian_stopwords)
# Fit the vectorizer and transform the text
#X = vectorizer.fit_transform(trainDf_final['question_text'] + " : : " + trainDf_final['document_plaintext'])


In [57]:
from tkinter.constants import X
from sklearn.model_selection import train_test_split
from sklearn import svm
from scipy.sparse import hstack

separator = ' <SEP> '

X_train_arabic = arab_vectorizer.fit_transform(arab_trainDf_final['question_text'] + separator + arab_trainDf_final['document_plaintext'])
y_train_arabic = arab_trainDf_final['oracle_result']
X_test_arabic = arab_vectorizer.transform(arab_validationDf_final['question_text'] + separator + arab_validationDf_final['document_plaintext'])
y_test_arabic = arab_validationDf_final['oracle_result']

X_train_bengali = bengali_vectorizer.fit_transform(bengali_trainDf_final['question_text'] + separator + bengali_trainDf_final['document_plaintext'])
y_train_bengali = bengali_trainDf_final['oracle_result']
X_test_bengali = bengali_vectorizer.transform(bengali_validationDf_final['question_text'] + separator + bengali_validationDf_final['document_plaintext'])
y_test_bengali = bengali_validationDf_final['oracle_result']

X_train_indonesian = indonesian_vectorizer.fit_transform(indonesian_trainDf_final['question_text'] + separator + indonesian_trainDf_final['document_plaintext'])
y_train_indonesian = indonesian_trainDf_final['oracle_result']
X_test_indonesian = indonesian_vectorizer.transform(indonesian_validationDf_final['question_text'] + separator + indonesian_validationDf_final['document_plaintext'])
y_test_indonesian = indonesian_validationDf_final['oracle_result']


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['একট', 'এব', 'কর', 'জন', 'রথম', 'হয'] not in stop_words.
  warnings.warn(


# SVM classifier

In [58]:
# Initialize and train the classifier
arabic_model = svm.SVC()
arabic_model.fit(X_train_arabic, y_train_arabic)

SVC()

In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred_arabic = arabic_model.predict(X_test_arabic)

# Calculate the performance metrics
print("Accuracy: ", accuracy_score(y_test_arabic, y_pred_arabic))
print("Precision: ", precision_score(y_test_arabic, y_pred_arabic))
print("Recall: ", recall_score(y_test_arabic, y_pred_arabic))
print("F1 Score: ", f1_score(y_test_arabic, y_pred_arabic))

Accuracy:  0.7581493165089379
Precision:  0.7975757575757576
Recall:  0.6919032597266036
F1 Score:  0.7409909909909911


In [60]:
bengali_model = svm.SVC()
bengali_model.fit(X_train_bengali, y_train_bengali)

SVC()

In [61]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred_bengali = bengali_model.predict(X_test_bengali)

# Calculate the performance metrics
print("Accuracy: ", accuracy_score(y_test_bengali, y_pred_bengali))
print("Precision: ", precision_score(y_test_bengali, y_pred_bengali))
print("Recall: ", recall_score(y_test_bengali, y_pred_bengali))
print("F1 Score: ", f1_score(y_test_bengali, y_pred_bengali))


Accuracy:  0.6785714285714286
Precision:  0.6538461538461539
Recall:  0.7589285714285714
F1 Score:  0.7024793388429752


In [62]:
indonesian_model = svm.SVC()
indonesian_model.fit(X_train_indonesian, y_train_indonesian)

SVC()

In [63]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred_indonesian = indonesian_model.predict(X_test_indonesian)

# Calculate the performance metrics
print("Accuracy: ", accuracy_score(y_test_indonesian, y_pred_indonesian))
print("Precision: ", precision_score(y_test_indonesian, y_pred_indonesian))
print("Recall: ", recall_score(y_test_indonesian, y_pred_indonesian))
print("F1 Score: ", f1_score(y_test_indonesian, y_pred_indonesian))


Accuracy:  0.7103274559193955
Precision:  0.7850678733031674
Recall:  0.5812395309882747
F1 Score:  0.6679499518768047
